In [1]:
## Implementation Framework for Team Formation problem
## Balancing Task Coverage vs. Maximum Expert Load
## Karan Vombatkere, Dec 2021

import numpy as np
import pandas as pd
import time

#Set logging level
import logging
logging.basicConfig(format='%(asctime)s |%(levelname)s: %(message)s', level=logging.INFO)

In [2]:
class TeamFormationProblem:
    '''
    Class to implement base framework for Team Formation problem
    The goal is to optimally Balance Task Coverage vs. Maximum Expert Load

    Parameters
    ----------
        m_tasks : list of m tasks; each task is a list of skills
        n_experts : list of n experts; each expert is a list of skills
    ----------
    self.taskAssignment  : Created (n x m) matrix to store task assignment
                            n rows represent the n experts, m columns are for m tasks
    '''

    def __init__(self, m_tasks, n_experts, max_workload_threshold=3):
        '''
        Initialize problem instance with m tasks, n experts
        ARGS:
            m_tasks : list of m tasks; each task is a list of skills
            n_experts : list of n experts; each expert is a list of skills
            max_workload_threshold: maximum workload threshold to attempt to solve for, default=3
        '''
        self.tasks = m_tasks
        self.m = len(self.tasks)

        self.experts = n_experts
        self.n = len(self.experts)

        self.maxWorkloadThreshold = max_workload_threshold

        #self.taskAssignment = np.zeros((self.n, self.m))
        logging.info('Team Formation Problem initialized with {} tasks and {} experts'.format(self.m,self.n))


    def getExpertsAssignedToTask(self, taskAssignment, task_index):
        '''
        Given a taskAssignment, and task index get the experts currently assigned to this task
        ARGS:
            taskAssignment  : current task assigment
            task_index  : index of task in original list m_tasks
        RETURN:
            assigned_experts_indices    : list of indices of experts assigned to task with index=task_index
        '''
        assigned_experts_arr = taskAssignment[:,task_index]
        assigned_experts_indices = []
        
        #Retrieve indices of experts assigned to task
        for expert_index, assigned in enumerate(assigned_experts_arr):
            if assigned == 1:
                assigned_experts_indices.append(expert_index)

        return assigned_experts_indices


    def getExpertSkillSet(self, expertIndices):
        '''
        Given a list of expert indices, return union of all expert skills
        ARGS:
            expertIndices   : list of indices of experts
        RETURN:
            expert_skillset    : Union of skills of all experts, returned as a set
        '''
        expert_skillset = set()
        for expert_index in expertIndices:
            expert_i_skills = set(self.experts[expert_index])
            expert_skillset = expert_skillset.union(expert_i_skills)

        return expert_skillset


    def singleTaskCoverage(self, taskAssignment, task_index):
        '''
        Given a taskAssignment, compute the task coverage of a single task 
        ARGS:
            taskAssignment  : current task assigment
            task_index  : index of task in original list m_tasks
        RETURN:
            task_coverage   : task coverage of a single task
        '''
        #Get union of skills of experts assigned to task
        task_assigned_experts_indices = self.getExpertsAssignedToTask(taskAssignment, task_index)
        all_expert_skills = self.getExpertSkillSet(task_assigned_experts_indices)

        task_skills = set(self.tasks[task_index])
        logging.debug("Single Task Coverage Debug: task index={}, task skills={}, expert skills={}".format(task_index, task_skills, all_expert_skills))
        
        #Compute coverage
        task_coverage = len(all_expert_skills.intersection(task_skills))/len(task_skills)
        
        return task_coverage



    def maximumExpertLoad(self, taskAssignment):
        '''
        Given a taskAssignment, compute the maximum expert load
        ARGS:
            taskAssignment  : current task assigment
        RETURN: 
            max_load    : Maximum expert workload
        '''
        max_load = 0

        for i in range(self.n):
            expert_i_tasks = taskAssignment[i,:]
            expert_i_load = np.sum(expert_i_tasks)
            
            if expert_i_load > max_load:
                max_load = expert_i_load
                
                if max_load == self.m:
                    return max_load

        return max_load



    def computeTotalTaskCoverage(self, taskAssigment = None, list_flag = True):
        '''
        Given a taskAssignment, compute the total task coverage of the assigment
        ARGS:
            taskAssignment  : current task assigment
            list_flag  : flag to determine whether to output total coverage or list
        RETURN:
            total_task_coverage_value   : total task coverage across all tasks
            total_task_coverage_list    : list of all m task coverage values
        '''
        if taskAssigment is not None:
            task_assigment = taskAssigment
        else:
            task_assigment = self.taskAssignment
        

        total_task_coverage_value = 0
        total_task_coverage_list = []

        for index_task, t in enumerate(self.tasks):
            cov = self.singleTaskCoverage(task_assigment, index_task)
            total_task_coverage_value += cov
            total_task_coverage_list.append(cov)

        if list_flag:
            return total_task_coverage_list
        
        else:
            return total_task_coverage_value



    def getBestExpertTaskEdge(self, taskAssignment, experts_array_indices, current_coverage_list):
        '''
        Greedily compute the best expert-task edge (assigment) that maximizes coverage in that iteration
        ARGS:
            taskAssignment : current task assigment
            experts_array_indices  : current list of available experts, as index values
            current_coverage_list    : current total task coverage of each task
        RETURN:
            max_edge_coverage  : maximum value of change in task coverage by adding edge (i,j)
            bestExpertTaskEdge  : indices of expert and task as a tuple (i,j)
        '''
        bestExpertTaskEdge = None
        max_edge_coverage = 0
        
        for i in experts_array_indices:
            E_i = self.experts[i]
            for j, T_j in enumerate(self.tasks):

                #Get experts currently assigned to T_j
                T_j_assigned_experts_indices = self.getExpertsAssignedToTask(taskAssignment, j)

                #Check if this expert is already assigned to T_j
                if i not in T_j_assigned_experts_indices:
                    #Compute union of skills of all experts assigned to T_j
                    expert_skills = self.getExpertSkillSet(T_j_assigned_experts_indices)
                    
                    expert_skills = expert_skills.union(set(E_i))       #Add expert E_i
                    task_skills = set(T_j)      #Get task skills as a set
                    
                    #Compute task coverage with expert added
                    T_j_coverage = len(expert_skills.intersection(task_skills))/len(task_skills)
                    delta_coverage = T_j_coverage - current_coverage_list[j]
                    logging.debug("deltaCoverage={:.1f}, T_j_coverage={}".format(delta_coverage, T_j_coverage))

                    if delta_coverage > max_edge_coverage:
                        max_edge_coverage = delta_coverage
                        bestExpertTaskEdge = (i,j)
                        
                        #Return immediately if all skills in a task are covered
                        if max_edge_coverage == 1:
                            return max_edge_coverage, bestExpertTaskEdge

        return max_edge_coverage, bestExpertTaskEdge


    def displayTaskAssignment(self, taskAssignment):
        '''
        Given a taskAssignment, print the assignment out for each expert
        ARGS:
            taskAssignment  : current task assigment
        '''
        for j in range(self.m):
            #Get experts assigned to T_j
            T_j_assigned_experts_indices = self.getExpertsAssignedToTask(taskAssignment, j)
            
            print("Experts Assigned to Task {}: {}".format(j, T_j_assigned_experts_indices))


    def greedyTaskAssignment(self, experts_arr_indices):
        '''
        Greedily compute a Task Assignment given a set of tasks and experts
        ARGS:
            experts_arr_indices  : list of available experts, as index values
        RETURN:
            taskAssignment  : greedy task assigment
        '''
        startTime = time.perf_counter()

        #Create empty task assigment matrix
        taskAssignment_i = np.zeros((self.n, self.m), dtype=np.int8)

        #Initialize current coverage list
        currentCoverageList = self.computeTotalTaskCoverage(taskAssignment_i)
        
        #Get next best assignment and coverage value
        deltaCoverage, best_ExpertTaskEdge = self.getBestExpertTaskEdge(taskAssignment_i, experts_arr_indices, currentCoverageList)

        #Assign edges until there is no more coverage possible or no experts left
        while (deltaCoverage != 0) and (experts_arr_indices):

            #Add edge to assignment
            taskAssignment_i[best_ExpertTaskEdge[0], best_ExpertTaskEdge[1]] = 1
            #self.displayTaskAssignment(taskAssignment_i)

            #Remove expert from experts_arr_indices
            experts_arr_indices.remove(best_ExpertTaskEdge[0])
            
            #Calculate current coverage list
            currentCoverageList = self.computeTotalTaskCoverage(taskAssignment_i)
            logging.debug("Current Coverage List = {}".format(currentCoverageList))

            #Get next best assignment and coverage value
            deltaCoverage, best_ExpertTaskEdge = self.getBestExpertTaskEdge(taskAssignment_i, experts_arr_indices, currentCoverageList)
            logging.debug("deltaCoverage={:.1f}, best_ExpertTaskEdge={}".format(deltaCoverage, best_ExpertTaskEdge))
        

        runTime = time.perf_counter() - startTime
        logging.debug("Greedy Task Assignment computation time = {:.1f} seconds".format(runTime))

        return taskAssignment_i


    def computeTaskAssigment(self):
        '''
        Compute a Task Assignment, of experts to tasks.
        Use m thresholds for the maximum load, and call a greedy method for each threshold
        Store this task assignment in self.taskAssignment
        '''
        startTime = time.perf_counter()
        F_max = 0 #store maximum objective value
        best_T_i = None

        for T_i in range(1, self.maxWorkloadThreshold+1):
            logging.info('--------------------------------------------------------------------------------------------------')
            logging.info("Computing Greedy Task Assignment for threshold max load, T_i={}".format(T_i))

            #Create T_i copies of each expert, using index values
            experts_index_list_T_i = [indx for indx in range(self.n)]*T_i
            logging.info("Expert Index List: {}".format(experts_index_list_T_i))

            #Greedily assign experts to tasks using this expert list
            taskAssignment_T_i = self.greedyTaskAssignment(experts_index_list_T_i)       
            logging.info("Greedy Task Assignment = \n{}".format(taskAssignment_T_i))

            #Compute Objective: Coverage - T_i
            F_i = self.computeTotalTaskCoverage(taskAssignment_T_i, list_flag=False) - T_i
            logging.info("F_i = {:.3f}".format(F_i))

            if F_i > F_max:
                F_max = F_i
                self.taskAssignment = taskAssignment_T_i
                best_T_i = T_i

        logging.info("Best Task Assignment is for max workload threshold: {}, F_i(max)={:.3f} \n{}".format(best_T_i, F_max, self.taskAssignment))
        
        runTime = time.perf_counter() - startTime
        logging.info("\nTotal Computation time = {:.3f} seconds".format(runTime))

        return None



In [3]:
#Preliminary Testing
task_list = [['s1', 's2', 's4', 's5'], ['s3', 's2'], ['s1', 's3', 's5'], ['s2', 's4'],['s1', 's4', 's5'], ['s3'], ['s1', 's5'], ['s2', 's4']]
expert_list = [['s2', 's3'], ['s2', 's4'], ['s3'], ['s5', 's4']]

In [4]:
teamFormationTest = TeamFormationProblem(task_list, expert_list)

2021-12-20 16:25:15,848 |INFO: Team Formation Problem initialized with 8 tasks and 4 experts


In [5]:
teamFormationTest.computeTaskAssigment()

2021-12-20 16:25:15,881 |INFO: --------------------------------------------------------------------------------------------------
2021-12-20 16:25:15,882 |INFO: Computing Greedy Task Assignment for threshold max load, T_i=1
2021-12-20 16:25:15,883 |INFO: Expert Index List: [0, 1, 2, 3]
2021-12-20 16:25:15,884 |INFO: Greedy Task Assignment = 
[[0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0]]
2021-12-20 16:25:15,885 |INFO: F_i = 2.667
2021-12-20 16:25:15,886 |INFO: --------------------------------------------------------------------------------------------------
2021-12-20 16:25:15,887 |INFO: Computing Greedy Task Assignment for threshold max load, T_i=2
2021-12-20 16:25:15,887 |INFO: Expert Index List: [0, 1, 2, 3, 0, 1, 2, 3]
2021-12-20 16:25:15,891 |INFO: Greedy Task Assignment = 
[[0 1 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 1 0 0 0]]
2021-12-20 16:25:15,892 |INFO: F_i = 3.500
2021-12-20 16:25:15,893 |INFO: -----------------------------

In [6]:
#Import real data
#Freelancer from DropBox link: https://www.dropbox.com/sh/8zpsi1etvvvvj5k/AAD-J9ZQmSsbnSmEILBMD9uxa/datasets/real?dl=0&subfolder_nav_tracking=1
#freelance_experts.csv and freelance_projects.csv

In [7]:
def extract_skills(row):
    skills = []
    for i,val in enumerate(row):
        if val == 1:
            skills+=str(i)
    return skills            

In [8]:
def importFreelancerData(experts_filename='../Datasets/freelance_experts.csv', tasks_filename='../Datasets/freelance_projects.csv'):
    #Extract tasks skills as list
    freelance_tasks_df = pd.read_csv(tasks_filename, header=None)
    freelance_tasks_df['Task_Skills'] = freelance_tasks_df.apply(lambda row: extract_skills(row), axis=1)
    task_skills_list = freelance_tasks_df.Task_Skills.to_list()
    
    #Extract experts skills as list
    freelance_experts_df = pd.read_csv(experts_filename, header=None)
    freelance_experts_df['Expert_Skills'] = freelance_experts_df.apply(lambda row: extract_skills(row), axis=1)
    expert_skills_list = freelance_experts_df.Expert_Skills.to_list()

    return task_skills_list, expert_skills_list

In [9]:
t,e = importFreelancerData()

In [12]:
FreelancerTest = TeamFormationProblem(t[0:300], e[0:200], max_workload_threshold=1)

2021-12-20 16:26:08,558 |INFO: Team Formation Problem initialized with 300 tasks and 200 experts


In [13]:
FreelancerTest.computeTaskAssigment()

2021-12-20 16:26:09,335 |INFO: --------------------------------------------------------------------------------------------------
2021-12-20 16:26:09,337 |INFO: Computing Greedy Task Assignment for threshold max load, T_i=1
2021-12-20 16:26:09,338 |INFO: Expert Index List: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 

In [15]:
FreelancerTest.displayTaskAssignment(FreelancerTest.taskAssignment)

Experts Assigned to Task 0: [128]
Experts Assigned to Task 1: [32]
Experts Assigned to Task 2: [12]
Experts Assigned to Task 3: [6]
Experts Assigned to Task 4: [41]
Experts Assigned to Task 5: []
Experts Assigned to Task 6: [129]
Experts Assigned to Task 7: []
Experts Assigned to Task 8: [70, 127]
Experts Assigned to Task 9: [132]
Experts Assigned to Task 10: [61]
Experts Assigned to Task 11: []
Experts Assigned to Task 12: [106, 164]
Experts Assigned to Task 13: [199]
Experts Assigned to Task 14: [155]
Experts Assigned to Task 15: [179]
Experts Assigned to Task 16: [45]
Experts Assigned to Task 17: [9]
Experts Assigned to Task 18: []
Experts Assigned to Task 19: []
Experts Assigned to Task 20: [110, 178]
Experts Assigned to Task 21: [63]
Experts Assigned to Task 22: [49]
Experts Assigned to Task 23: [66]
Experts Assigned to Task 24: [34]
Experts Assigned to Task 25: []
Experts Assigned to Task 26: [18]
Experts Assigned to Task 27: []
Experts Assigned to Task 28: []
Experts Assigned to